## Introduction and Schema Diagram

In [1]:
%%capture
%load_ext sql
%sql sqlite:////Users/conq_/database/DQProjects/chinook.db

## Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:////Users/conq_/database/DQProjects/chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


##  Selecting New Albums to Purchase

In [3]:
%%sql

WITH usa_tracks_sold AS
   (
    SELECT il.* FROM invoice_line il
    INNER JOIN invoice i on il.invoice_id = i.invoice_id
    INNER JOIN customer c on i.customer_id = c.customer_id
    WHERE c.country = "USA"
   )

SELECT
    g.name genre,
    count(uts.invoice_line_id) tracks_sold,
    ROUND((cast(count(uts.invoice_line_id) AS FLOAT) / (
        SELECT COUNT(*) from usa_tracks_sold
    ))*100, 1) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t on t.track_id = uts.track_id
INNER JOIN genre g on g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC


 * sqlite:////Users/conq_/database/DQProjects/chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Pop,22,2.1
Latin,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

- Red Tone (Punk)
- Slim Jim Bites (Blues)
- Meteor and the Girls (Pop)
It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

## Analyzing Employee Sales Performance

In [4]:
%%sql
WITH cs AS
    (
        SELECT 
    c.first_name || ' ' || c.last_name customer_name,
    SUM(i.total) customer_spent,
    c.support_rep_id emp_id
    FROM customer c
    LEFT JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY 1
    ORDER BY 1
    )
SELECT
    e.first_name || ' ' || e.last_name employee_name,
    ROUND(SUM(customer_spent), 2) revenue_generated,
    e.hire_date hired_at
    FROM cs
    INNER JOIN employee e ON e.employee_id = cs.emp_id
    GROUP BY 1 
    ORDER BY 1

 * sqlite:////Users/conq_/database/DQProjects/chinook.db
Done.


employee_name,revenue_generated,hired_at
Jane Peacock,1731.51,2017-04-01 00:00:00
Margaret Park,1584.0,2017-05-03 00:00:00
Steve Johnson,1393.92,2017-10-17 00:00:00


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analyzing Sales by Country

In [5]:
%%sql
WITH cc AS
(
  SELECT 
    CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
    COUNT(c.first_name || ' ' || c.last_name) order_count,
    SUM(i.total) total_spent
    FROM customer c
    LEFT JOIN invoice i ON i.customer_id = c.customer_id
    GROUP BY 1
    ORDER BY 1
),
cu AS
(
SELECT
    CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
    COUNT(c.first_name || ' ' || c.last_name) customer_count
    FROM customer c
    GROUP BY 1
    ORDER BY 1
)

SELECT 
    cu.country,
    cu.customer_count,
    ROUND(cc.total_spent, 2) total_sales,
    ROUND(CAST(cc.total_spent AS FLOAT)/cu.customer_count, 2) avg_sales_per_customer,
    ROUND(CAST(cc.total_spent AS FLOAT)/cc.order_count, 2) avg_order
    FROM (
        SELECT cu.*,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
        FROM cu
    ) cu
    LEFT JOIN cc ON cu.country = cc.country
    GROUP BY 1
    ORDER BY sort

 * sqlite:////Users/conq_/database/DQProjects/chinook.db
Done.


country,customer_count,total_sales,avg_sales_per_customer,avg_order
Brazil,5,427.68,85.54,7.01
Canada,8,535.59,66.95,7.05
Czech Republic,2,273.24,136.62,9.11
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
India,2,183.15,91.57,8.72
Portugal,2,185.13,92.57,6.38
USA,13,1040.49,80.04,7.94
United Kingdom,3,245.52,81.84,8.77
Other,15,1094.94,73.0,7.45


Based on the data, there may be opportunity in the following countries:

- Czech Republic
- United Kingdom
- India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.

## Albums vs Individual Tracks

In [6]:
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;

 * sqlite:////Users/conq_/database/DQProjects/chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,0.8143322475570033
yes,114,0.18566775244299674


Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.